# Feathr Quick Start Notebook

This notebook illustrates the use of Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

The major problems Feathr solves are:

1. Create, share and manage useful features from raw source data.
2. Provide Point-in-time feature join to create training dataset to ensure no data leakage.
3. Deploy the same feature data to online store to eliminate training and inference data skew.

## Prerequisite

Feathr has native cloud integration. First step is to provision required cloud resources if you want to use Feathr.

Follow the [Feathr ARM deployment guide](https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-arm.html) to run Feathr on Azure. This allows you to quickly get started with automated deployment using Azure Resource Manager template. For more details, please refer [README.md](https://github.com/feathr-ai/feathr#%EF%B8%8F-running-feathr-on-cloud-with-a-few-simple-steps).

Additionally, to run this notebook, you'll need to install `feathr` pip package. For local spark, simply run `pip install feathr` on the machine that runs this notebook. To use Databricks or Azure Synapse Analytics, please see dependency management documents:
- [Azure Databricks dependency management](https://learn.microsoft.com/en-us/azure/databricks/libraries/)
- [Azure Synapse Analytics dependency management](https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-azure-portal-add-libraries)

## Notebook Steps

This tutorial demonstrates the key capabilities of Feathr, including:

1. Install Feathr and necessary dependencies
2. Create shareable features with Feathr feature definition configs
3. Create training data using point-in-time correct feature join
4. Train a prediction model and evaluate the features
5. Register the features to share across teams
6. Materialize feature values for online scoring

The overall data flow is as follows:

<img src="https://github.com/linkedin/feathr/blob/main/docs/images/feature_flow.png?raw=true" width="800">

## 1. Install Feathr and Necessary Dependancies

Run the following cells if you haven't installed `feathr` package already. 

In [ ]:
!pip install feathr

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import glob
import json
from math import sqrt
import os
import requests
from tempfile import NamedTemporaryFile

from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from feathr import (
    FeathrClient,
    # Feature data types
    BOOLEAN, FLOAT, INT32, ValueType,
    # Feature data sources
    INPUT_CONTEXT, HdfsSource,
    # Feature aggregations
    TypedKey, WindowAggTransformation,
    # Feature types and anchor
    DerivedFeature, Feature, FeatureAnchor,
    # Materialization
    BackfillTime, MaterializationSettings, RedisSink,
    # Offline feature computation
    FeatureQuery, ObservationSettings,
)
from feathr.spark_provider.feathr_configurations import SparkExecutionConfiguration
from feathr.utils.job_utils import get_result_df

import pandas as pd
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

## 2. Create Shareable Features with Feathr Feature Definition Configs

First, we define all the necessary resource key values for authentication. These values can also be retrieved by using [Azure Key Vault](https://azure.microsoft.com/en-us/services/key-vault/) cloud key value store.
Please refer to [A note on using azure key vault to store credentials](https://github.com/feathr-ai/feathr/blob/41e7496b38c43af6d7f8f1de842f657b27840f6d/docs/how-to-guides/feathr-configuration-and-env.md#a-note-on-using-azure-key-vault-to-store-credentials) for more details.

In [3]:
RESOURCE_PREFIX = "juntest"
PROJECT_NAME = "feathr_getting_started"
ROOT_PATH = "./"  # Could be Azure Blob File System path, abfs or wasbs too.

KEY_VAULT = f"{RESOURCE_PREFIX}kv"
KEY_VAULT_URI = f"https://{KEY_VAULT}.vault.azure.net"

In [4]:
# TODO clean up
# synapse_workspace_url=resource_prefix+"syws"
# adls_account=resource_prefix+"dls"
# adls_fs_name=resource_prefix+"fs"
# purview_name=resource_prefix+"purview"

# # Set the resource link
# os.environ['spark_config__azure_synapse__dev_url'] = f'https://{synapse_workspace_url}.dev.azuresynapse.net'
# os.environ['spark_config__azure_synapse__pool_name'] = 'spark31'
# os.environ['spark_config__azure_synapse__workspace_dir'] = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/feathr_project'
# feathr_output_path = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windo

In [5]:
# Get all the required credentials from Azure Key Vault
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
secret_client = SecretClient(vault_url=KEY_VAULT_URI, credential=credential)
retrieved_secret = secret_client.get_secret('FEATHR-ONLINE-STORE-CONN').value

To run this notebook on **Azure Synapse** or **Local Spark**, you'll need to set `AZURE_CLIENT_ID`, `AZURE_TENANT_ID`, `AZURE_CLIENT_SECRET`, and `REDIS_PASSWORD` environment variables.

To run this notebook on **Databricks**, you'll need to set `DATABRICKS_WORKSPACE_TOKEN_VALUE` and `REDIS_PASSWORD`.

In [6]:
# Redis credential
os.environ['REDIS_PASSWORD'] = retrieved_secret.split(",")[1].split("password=", 1)[1]

# To use a local spark:
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"


# To use a synapse Spark pool:
# from pyspark.sql import SparkSession
# sc = SparkSession.builder.getOrCreate()
# token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary
# connection_string = token_library.GetSecret("<AZURE KEY VAULT NAME>", "<SECRET KEY>", "<LINKED SERVICE NAME>")

# To use a databricks cluster:
# ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
# databricks_config = {
#     'run_name': "FEATHR_FILL_IN",
#     'existing_cluster_id': ctx.tags().get('clusterId').get(),
#     'libraries': [{'jar': "FEATHR_FILL_IN"}],
#     'spark_jar_task': {
#         'main_class_name': "FEATHR_FILL_IN",
#         'parameters': ["FEATHR_FILL_IN"],
#     },
# }
# os.environ['spark_config__spark_cluster'] = "databricks"
# os.environ['spark_config__databricks__workspace_instance_url'] = "https://" + ctx.tags().get('browserHostName').get()
# os.environ['spark_config__databricks__config_template'] = json.dumps(databricks_config)
# os.environ['spark_config__databricks__work_dir'] = "dbfs:/feathr_getting_started"
# os.environ['DATABRICKS_WORKSPACE_TOKEN_VALUE'] = ctx.apiToken().get()


### Configurations

Feathr uses a yaml file to define configurations. Please refer to [feathr_config.yaml]( https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) for the meaning of each field.

In [7]:
yaml_config = f"""
api_version: 1

project_config:
  project_name: {PROJECT_NAME}
  required_environment_variables:
    - 'REDIS_PASSWORD'
    - 'AZURE_CLIENT_ID'
    - 'AZURE_TENANT_ID'
    - 'AZURE_CLIENT_SECRET'
    
feature_registry:
  api_endpoint: 'https://{RESOURCE_PREFIX}webapp.azurewebsites.net/api/v1'
  # Registry configs if use purview - TODO do I need this even when using api_endpoint?
  #purview:
    #purview_name: "feathrazuretest3-purview1"
    # delimiter indicates that how the project/workspace name, feature names etc. are delimited. By default it will be '__'
    # this is for global reference (mainly for feature sharing). For example, when we setup a project called foo, and we have an anchor called 'taxi_driver' and the feature name is called 'f_daily_trips'
    # the feature will have a globally unique name called 'foo__taxi_driver__f_daily_trips'
    #delimiter: "__"
    # controls whether the type system will be initialized or not. Usually this is only required to be executed once.
    #type_system_initialization: false

spark_config:
  # Currently support: 'azure_synapse', 'databricks', and 'local'
  spark_cluster: 'local'
  spark_result_output_parts: '1'

offline_store:
  wasb:
    wasb_enabled: true

online_store:
  # You can skip this part if you don't have Redis and skip materialization later in this notebook.
  redis:
    host: '{RESOURCE_PREFIX}redis.redis.cache.windows.net'
    port: 6380
    ssl_enabled: true
"""

tmp = NamedTemporaryFile(mode='w', delete=False)
with open(tmp.name, "w") as config_file:
    config_file.write(yaml_config)

All the configurations can be overwritten by environment variables with concatenation of `__` for different layers of the config file. For example, `feathr_runtime_location` for databricks config can be overwritten by setting `spark_config__databricks__feathr_runtime_location` environment variable.

### Initialize Feathr client

In [8]:
client = FeathrClient(config_path=tmp.name)

2022-10-05 17:51:56.623 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - secrets__azure_key_vault__name not found in the config file.
2022-10-05 17:51:56.640 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__s3__s3_enabled not found in the config file.
2022-10-05 17:51:56.643 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__adls__adls_enabled not found in the config file.
2022-10-05 17:51:56.649 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__jdbc__jdbc_enabled not found in the config file.
2022-10-05 17:51:56.653 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__snowflake__snowflake_enabled not found in the config file.
2022-10-05 17:51:56.662 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - spark_config__loc

### Prepare the NYC taxi fare dataset

In [156]:
DATA_FILEPATH = ROOT_PATH + "green_tripdata_2020-04_with_index.csv"

# Download the data file
response = requests.get(
    "https://azurefeathrstorage.blob.core.windows.net/public/sample_data/green_tripdata_2020-04_with_index.csv",
)
with open(DATA_FILEPATH, "wb") as data_file:
    data_file.write(response.content)

df = spark.read.option("header", True).csv(DATA_FILEPATH)
df.limit(5).toPandas()

,trip_id,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,0,2.0,2020-04-01 00:44:02,2020-04-01 00:52:23,N,1.0,42,41,1.0,1.68,...,0.5,0.5,0.0,0.0,None,0.3,9.3,1.0,1.0,0.0
1,1,2.0,2020-04-01 00:24:39,2020-04-01 00:33:06,N,1.0,244,247,2.0,1.94,...,0.5,0.5,0.0,0.0,None,0.3,10.3,2.0,1.0,0.0
2,2,2.0,2020-04-01 00:45:06,2020-04-01 00:51:13,N,1.0,244,243,3.0,1.0,...,0.5,0.5,0.0,0.0,None,0.3,7.8,2.0,1.0,0.0
3,3,2.0,2020-04-01 00:45:06,2020-04-01 01:04:39,N,1.0,244,243,2.0,2.81,...,0.5,0.5,0.0,0.0,None,0.3,13.3,2.0,1.0,0.0
4,4,2.0,2020-04-01 00:00:23,2020-04-01 00:16:13,N,1.0,75,169,1.0,6.79,...,0.5,0.5,0.0,0.0,None,0.3,22.3,1.0,1.0,0.0


### Defining features with Feathr

In Feathr, a feature is viewed as a function, mapping a key and timestamp to a feature value. For more details, please see [Feathr Feature Definition Guide](https://github.com/feathr-ai/feathr/blob/main/docs/concepts/feature-definition.md).

* The feature key (a.k.a. entity id) identifies the subject of feature, e.g. a user_id or location_id.
* The feature name is the aspect of the entity that the feature is indicating, e.g. the age of the user.
* The feature value is the actual value of that aspect at a particular time, e.g. the value is 30 at year 2022.

Note that, in some cases, a feature could be just a transformation function that has no entity key or timestamp involved, e.g. *the day of week of the request timestamp*.

There are two types of features -- anchored features and derivated features:

* **Anchored features**: Features that are directly extracted from sources. Could be with or without aggregation. 
* **Derived features**: Features that are computed on top of other features.

#### Define anchored features

A feature source is needed for anchored features that describes the raw data in which the feature values are computed from. A source value should be either `INPUT_CONTEXT` (the features that will be extracted from the observation data directly) or `feathr.source.Source` object.

In [10]:
TIMESTAMP_COL = "lpep_dropoff_datetime"
TIMESTAMP_FORMAT = "yyyy-MM-dd HH:mm:ss"

In [11]:
# We define f_trip_distance and f_trip_time_duration features separately
# so that we can reuse them later for the derived features.
f_trip_distance = Feature(
    name="f_trip_distance",
    feature_type=FLOAT,
    transform="trip_distance",
)
f_trip_time_duration = Feature(
    name="f_trip_time_duration",
    feature_type=FLOAT,
    transform="cast_float((to_unix_timestamp(lpep_dropoff_datetime) - to_unix_timestamp(lpep_pickup_datetime)) / 60)",
)

features = [
    f_trip_distance,
    f_trip_time_duration,
    Feature(
        name="f_is_long_trip_distance",
        feature_type=BOOLEAN,
        transform="trip_distance > 30.0",
    ),
    Feature(
        name="f_day_of_week",
        feature_type=INT32,
        transform="dayofweek(lpep_dropoff_datetime)",
    ),
    Feature(
        name="f_day_of_month",
        feature_type=INT32,
        transform="dayofmonth(lpep_dropoff_datetime)",
    ),
    Feature(
        name="f_hour_of_day",
        feature_type=INT32,
        transform="hour(lpep_dropoff_datetime)",
    ),
]

# After you have defined features, bring them together to build the anchor to the source.
feature_anchor = FeatureAnchor(
    name="feature_anchor",
    source=INPUT_CONTEXT,  # Pass through source, i.e. observation data.
    features=features,
)

We can define the source with a preprocessing python function.

In [12]:
def preprocessing(df: DataFrame) -> DataFrame:
    import pyspark.sql.functions as F
    df = df.withColumn("fare_amount_cents", (F.col("fare_amount") * 100.0).cast("float"))
    return df

batch_source = HdfsSource(
    name="nycTaxiBatchSource",
    path=DATA_FILEPATH,
    event_timestamp_column=TIMESTAMP_COL,
    preprocessing=preprocessing,
    timestamp_format=TIMESTAMP_FORMAT,
)

For the features with aggregation, the supported functions are as follows:

| Aggregation Function | Input Type | Description |
| --- | --- | --- |
|SUM, COUNT, MAX, MIN, AVG	|Numeric|Applies the the numerical operation on the numeric inputs. |
|MAX_POOLING, MIN_POOLING, AVG_POOLING	| Numeric Vector | Applies the max/min/avg operation on a per entry bassis for a given a collection of numbers.|
|LATEST| Any |Returns the latest not-null values from within the defined time window |

In [13]:
agg_key = TypedKey(
    key_column="DOLocationID",
    key_column_type=ValueType.INT32,
    description="location id in NYC",
    full_name="nyc_taxi.location_id",
)

agg_window = "90d"

# Anchored features with aggregations
agg_features = [
    Feature(
        name="f_location_avg_fare",
        key=agg_key,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="fare_amount_cents",
            agg_func="AVG",
            window=agg_window,
        ),
    ),
    Feature(
        name="f_location_max_fare",
        key=agg_key,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="fare_amount_cents",
            agg_func="MAX",
            window=agg_window,
        ),
    ),
]

agg_feature_anchor = FeatureAnchor(
    name="agg_feature_anchor",
    source=batch_source,  # External data source for feature. Typically a data table.
    features=agg_features,
)

#### Define derived features

We also define a derived feature, `f_trip_time_distance`, from the anchored features `f_trip_distance` and `f_trip_time_duration` as follows:

In [14]:
derived_features = [
    DerivedFeature(
        name="f_trip_time_distance",
        feature_type=FLOAT,
        input_features=[
            f_trip_distance,
            f_trip_time_duration,
        ],
        transform="f_trip_distance / f_trip_time_duration",
    )
]

### Build features

Finally, we build the features.

In [15]:
client.build_features(
    anchor_list=[feature_anchor, agg_feature_anchor],
    derived_feature_list=derived_features,
)

## 3. Create Training Data Using Point-in-Time Correct Feature Join

After the feature producers have defined the features (as described in the Feature Definition part), the feature consumers may want to consume those features. Feature consumers will use observation data to query from different feature tables using Feature Query.

To create a training dataset using Feathr, one needs to provide a feature join configuration file to specify
what features and how these features should be joined to the observation data. 

To learn more on this topic, please refer to [Point-in-time Correctness](https://github.com/linkedin/feathr/blob/main/docs/concepts/point-in-time-join.md)

In [39]:
feature_names = [feature.name for feature in features + agg_features + derived_features]
feature_names

['f_trip_distance',
 'f_trip_time_duration',
 'f_is_long_trip_distance',
 'f_day_of_week',
 'f_day_of_month',
 'f_hour_of_day',
 'f_location_avg_fare',
 'f_location_max_fare',
 'f_trip_time_distance']

In [20]:
DATA_FORMAT = "avro"  # or "parquet"
offline_features_path = ROOT_PATH + f"feathr_output.{DATA_FORMAT}"

In [21]:
# Features that we want to request. Can use a subset of features
query = FeatureQuery(
    feature_list=feature_names,
    key=agg_key,
)
settings = ObservationSettings(
    observation_path=DATA_FILEPATH,
    event_timestamp_column=TIMESTAMP_COL,
    timestamp_format=TIMESTAMP_FORMAT,
)
client.get_offline_features(
    observation_settings=settings,
    feature_query=query,
    # TODO - this doesn't work. It keeps storing as "avro" files
    execution_configurations=SparkExecutionConfiguration({
        "spark.feathr.inputFormat": DATA_FORMAT,
        "spark.feathr.outputFormat": DATA_FORMAT,
    }),
    output_path=offline_features_path,
)

client.wait_job_to_finish(timeout_sec=500)

2022-10-05 17:52:16.772 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - BLOB_ACCOUNT is not set in the environment variables.
2022-10-05 17:52:16.773 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - BLOB_KEY is not set in the environment variables.
2022-10-05 17:52:16.774 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:75 - Local Spark Mode only support basic params right now and should be used only for testing purpose.
2022-10-05 17:52:16.776 | INFO     | feathr.spark_provider._localspark_submission:_init_args:200 - Spark job: feathr_getting_started_feathr_feature_join_job is running on local spark with master: local[*].
2022-10-05 17:52:16.779 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:120 - Detail job stdout and stderr are in debug/feathr_getting_started_feathr_feature_join_job20221005175216/log.
2022-10-05 17:52:16.780 | INFO     | feathr.spark_provider._localspark_submissi

debug/feathr_getting_started_feathr_feature_join_job20221005175216
['/tmp/tmppe35xkpj/feathr_pyspark_driver.py']
x>>>>>>>>>>>>>>>>>>>>>>>>>>>>

2022-10-05 17:52:51.827 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:183 - Spark job with pid 3132133 finished in: 35 seconds with returncode 0


>

In [31]:
# Show feature results
get_result_df(
    spark=spark,
    client=client,
    data_format=DATA_FORMAT, res_url=offline_features_path,
).select(feature_names).limit(5).toPandas()

,f_trip_distance,f_trip_time_duration,f_is_long_trip_distance,f_day_of_week,f_day_of_month,f_hour_of_day,f_location_avg_fare,f_location_max_fare,f_trip_time_distance
0,10.06,16.000000,False,4,1,3,2266.000000,2266.0,0.628750
1,10.70,29.000000,False,4,1,6,3043.000000,3820.0,0.368966
2,5.19,7.983333,False,4,1,7,2545.333252,3820.0,0.650104
3,5.26,18.683332,False,4,1,7,2396.500000,3820.0,0.281534
4,6.26,16.733334,False,4,1,7,2327.199951,3820.0,0.374104


## 4. Train a Prediction Model and Evaluate the Features

After generating all the features, we train and evaluate a machine learning model to predict the NYC taxi fare prediction. In this example, we use Spark MLlib's [GBTRegressor](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-regression)

In [70]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor


# To run on a local spark, start a spark session:
from pyspark.sql import SparkSession
spark = (
    SparkSession
    .builder
    .appName("feathr")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.0")
    .config("spark.ui.port", "8080")  # Set ui port other than the default one (4040) so that feathr spark job doesn't fail. 
    .getOrCreate()
)

### Load Train and Test Data from the Offline Feature Values

In [71]:
# Train / test split
train_df, test_df = (
    spark.read.format(DATA_FORMAT).load(offline_features_path)
    .withColumn("label", F.col("fare_amount").cast("double"))
    .where(F.col("f_trip_time_duration") > 0)
    .fillna(0)
    .randomSplit([0.8, 0.2])
)

train_df.limit(5).toPandas()

,trip_id,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,f_is_long_trip_distance,f_day_of_month,f_trip_time_duration,f_day_of_week,f_hour_of_day,f_trip_distance,f_location_max_fare,f_location_avg_fare,f_trip_time_distance,label
0,0,2.0,2020-04-01 00:44:02,2020-04-01 00:52:23,N,1.0,42,41,1.0,1.68,...,False,1,8.350000,4,0,1.68,973.0,824.333313,0.201198,8.0
1,10,2.0,2020-04-01 00:18:32,2020-04-01 00:29:08,N,1.0,130,205,1.0,3.41,...,False,1,10.600000,4,0,3.41,1200.0,1200.000000,0.321698,12.0
2,100,2.0,2020-04-01 07:26:09,2020-04-01 07:29:26,N,1.0,74,74,1.0,0.79,...,False,1,3.283333,4,7,0.79,5537.0,1322.888916,0.240609,-5.0
3,1000,2.0,2020-04-01 19:03:14,2020-04-01 19:04:26,N,1.0,75,75,1.0,0.42,...,False,1,1.200000,4,19,0.42,8950.0,1149.923096,0.350000,3.5
4,10004,1.0,2020-04-12 18:20:20,2020-04-12 18:37:07,N,1.0,52,140,1.0,0.0,...,False,12,16.783333,1,18,0.00,6013.0,1913.714233,0.000000,23.7


In [72]:
print(f"Num train samples: {train_df.count()}")
print(f"Num test samples: {test_df.count()}")

Num train samples: 28486
Num test samples: 7050


### Build a ML Pipeline

In [77]:
# Generate a feature vector column for SparkML
vector_assembler = VectorAssembler(
    inputCols=[x for x in df.columns if x in feature_names],
    outputCol="features",
)

# Define a model
gbt = GBTRegressor(
    featuresCol="features",
    maxIter=100,
    maxDepth=5,
    maxBins=16,
)

# Create a ML pipeline
ml_pipeline = Pipeline(stages=[
    vector_assembler,
    gbt,
])

### Train and Evaluate the Model

In [78]:
# Train a model
model = ml_pipeline.fit(train_df)

# Make predictions
predictions = model.transform(test_df)

# Evaluate
evaluator = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse",
)
rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 5.39574


## 5. Register the Features to Share Across Teams

You can register your features in the centralized registry and share the corresponding project with other team members who want to consume those features and for further use.

In [74]:
client.register_features()
client.list_registered_features(project_name=PROJECT_NAME)  # TODO can I get other project's features than client's?

2022-10-05 23:35:54.795 | INFO     | feathr.registry._feathr_registry_client:register_features:126 - Check project lineage by this link: https://juntestwebapp.azurewebsites.net/projects/feathr_getting_started/lineage


[{'name': 'f_day_of_week',
  'id': '2a88dc72-51b1-49a8-9c3e-b08d148d199f',
  'qualifiedName': 'feathr_getting_started__feature_anchor__f_day_of_week'},
 {'name': 'f_location_avg_fare',
  'id': '4c0251ed-8d21-44b6-981b-e1a7f63d27aa',
  'qualifiedName': 'feathr_getting_started__agg_feature_anchor__f_location_avg_fare'},
 {'name': 'f_hour_of_day',
  'id': '649119e7-6242-48c8-863d-25c3a106ba2f',
  'qualifiedName': 'feathr_getting_started__feature_anchor__f_hour_of_day'},
 {'name': 'f_day_of_month',
  'id': '7a06eb7c-ec44-4dcf-a831-56d94b323276',
  'qualifiedName': 'feathr_getting_started__feature_anchor__f_day_of_month'},
 {'name': 'f_trip_time_distance',
  'id': 'ad484268-6d99-457e-bc1c-e2863f23fc90',
  'qualifiedName': 'feathr_getting_started__f_trip_time_distance'},
 {'name': 'f_trip_distance',
  'id': 'd6553d1f-efb6-45ed-bbed-4c3d57e2a055',
  'qualifiedName': 'feathr_getting_started__feature_anchor__f_trip_distance'},
 {'name': 'f_location_max_fare',
  'id': 'd67299f0-c3fd-4db8-9a36-08

## 6. Materialize Feature Values for Online Scoring

While we computed feature values on-the-fly at request time via Feathr, we can pre-compute the feature values and materialize them to offline or online storages such as Redis.

Note, only the features anchored to offline data source can be materialized, in our case `agg_features`.

In [75]:
# Get registered features
registered_features_dict = client.get_features_from_registry(PROJECT_NAME)

# TODO easier way to get this? since we'll need to use this list to materialize later.
agg_feature_names = []
for feature_name, feature in registered_features_dict.items():
    if feature.key[0].key_column != "NOT_NEEDED":
        agg_feature_names.append(feature_name)
agg_feature_names

['f_location_avg_fare', 'f_location_max_fare']

In [157]:
last_timestamp = (
    df
    .select(F.to_timestamp(F.col(TIMESTAMP_COL), TIMESTAMP_FORMAT).alias(TIMESTAMP_COL))
    .agg({TIMESTAMP_COL: "max"})
    .collect()[0][0]
)
last_timestamp

datetime.datetime(2020, 5, 1, 0, 47)

In [109]:
FEATURE_TABLE_NAME = "nycTaxiDemoFeature"

# Time range to materialize -- TODO how to properly set this?
backfill_time = BackfillTime(
    start=last_timestamp,
    end=last_timestamp,
    step=timedelta(days=1),
)

# Destinations:
# For online store,
redis_sink = RedisSink(table_name=FEATURE_TABLE_NAME)

# For offline store,
# adls_sink = HdfsSink(output_path=f"abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/{FEATURE_TABLE_NAME}/")

settings = MaterializationSettings(
    name=FEATURE_TABLE_NAME + ".job",  # job name
    backfill_time=backfill_time,
    sinks=[redis_sink],  # and/or adls_sink -- TODO can I specify both at the same time?
    feature_names=agg_feature_names,  # TODO can i pass the features of different keys together?
)

client.materialize_features(
    settings=settings,
    execution_configurations={"spark.feathr.outputFormat": "parquet"},
)

client.wait_job_to_finish(timeout_sec=500)

2022-10-06 00:24:05.971 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - KAFKA_SASL_JAAS_CONFIG is not set in the environment variables.
2022-10-06 00:24:05.972 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - BLOB_ACCOUNT is not set in the environment variables.
2022-10-06 00:24:05.972 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - BLOB_KEY is not set in the environment variables.
2022-10-06 00:24:05.976 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - monitoring__database__sql__url not found in the config file.
2022-10-06 00:24:05.979 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - monitoring__database__sql__user not found in the config file.
2022-10-06 00:24:05.981 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - MONITORING_DATABASE_SQL_PASSWORD is not set in the environment variables.
2022-10-06 00:24:05.981 | W

debug/feathr_getting_started_feathr_feature_materialization_job20221006002405
['/tmp/tmppe35xkpj/feathr_pyspark_driver.py']
x>>>>>>>>>>>>>>

2022-10-06 00:24:27.013 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:163 - Pyspark job Completed


>

2022-10-06 00:24:28.014 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:183 - Spark job with pid 3204258 finished in: 22 seconds with returncode 143


>

In [158]:
samples_df = df.where(F.col(TIMESTAMP_COL) == last_timestamp)
samples_df.toPandas()

,trip_id,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,35592,None,2020-04-30 23:51:00,2020-05-01 00:47:00,None,None,159,208,None,24.95,...,0.0,0.0,0.0,0.0,None,0.3,26.43,None,None,None


In [159]:
keys = samples_df.select(agg_key.key_column).distinct().toPandas()[agg_key.key_column].to_list()
keys

['208']

In [160]:
# Note, to get a single key, you may use client.get_online_features
agg_feature_values = client.multi_get_online_features(
    feature_table=FEATURE_TABLE_NAME,
    keys=keys,
    feature_names=agg_feature_names,
)
agg_feature_values

{'208': [1846.0, 6235.0]}

In [161]:
agg_feature_pdf = (
    pd.DataFrame
    .from_dict(agg_feature_values, orient='index', columns=agg_feature_names)
    .reset_index(names=agg_key.key_column)
)
agg_feature_pdf

,DOLocationID,f_location_avg_fare,f_location_max_fare
0,208,1846.0,6235.0


In [ ]:
# TODO feature gap: Cannot easily extract all the other features and join w/ the online-features to make online prediction.

In [69]:
# Stop the spark session if it is a local session.
spark.stop()